In [13]:
import torch
import torchvision
import torch.nn as nn
import numpy as np
import torchvision.transforms as TF

ModuleNotFoundError: No module named 'torch.utils.data.Dataset'

### 1. Basic autograd example 1   

In [2]:
# Create Tensor
x = torch.tensor(1., requires_grad=True)
w = torch.tensor(2., requires_grad=True)
b = torch.tensor(3., requires_grad=True)

# Build a computational graph
y = w * x + b # y = 2 * x + 3

# Compute Gradients
y.backward()

# get value
print(x.grad)
print(w.grad)
print(b.grad)

tensor(2.)
tensor(1.)
tensor(1.)


### 2. Basic autograd example 2

In [3]:
# Create tensor of shape(10, 3) as (10, 2)
x = torch.randn(10, 3)
y = torch.randn(10, 2)

# print(x.data)
# print(y.data)

linear = nn.Linear(3, 2)
print('w: ', linear.weight)
print('b: ', linear.bias)

# Build loss function and optimizer
criterion = nn.MSELoss()
optimizer = torch.optim.SGD(linear.parameters(), lr=0.01)

# Forward pass
pred = linear(x)

# Compute loss
print('dL/dw = ', linear.weight.grad)
print('dL/db = ', linear.bias.grad)

# 1-step gradient descent
optimizer.step()

# print out loss
pred = linear(x)
loss = criterion(pred, y)

print('Loss = ', loss.item())


w:  Parameter containing:
tensor([[ 0.5376,  0.1540,  0.3469],
        [-0.0983, -0.0502, -0.5432]], requires_grad=True)
b:  Parameter containing:
tensor([ 0.3768, -0.2052], requires_grad=True)
dL/dw =  None
dL/db =  None
Loss =  2.5665335655212402


### 3. Loading data from numpy

In [4]:
x = np.array([
    [1, 2],
    [3, 4]
])
y = torch.from_numpy(x)
z = y.numpy()

print(x)
print(y)
print(z)

[[1 2]
 [3 4]]
tensor([[1, 2],
        [3, 4]], dtype=torch.int32)
[[1 2]
 [3 4]]


### 4. Input pipeline

In [21]:
# Download and construct cifar-10 dataset
train_dataset = torchvision.datasets.CIFAR10(
    root='./data/',
    download=True,
    transform=TF.ToTensor(),
    train=True
)

image, label = train_dataset[0]
print(image.size())
print(label)

# Data loader
train_loader = torch.utils.data.DataLoader(
    dataset=train_dataset,
    batch_size=64,
    shuffle=True
)

# When iter starts, load data from files
data_iter = iter(train_loader)

# mini batch images and labels
images, labels = data_iter.next()

# Actual usage of the dataloader 
for images, labels in train_loader:
    # Training code should be written here
    pass

Files already downloaded and verified
torch.Size([3, 32, 32])
6


### 5. Input pipeline for custom dataset

In [22]:
# You should build your custom dataset ad below
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self):
        # 1. Initial file paths or a list of file names
        pass
    
    def __getitem__(self, index):
        # 1. Read one data from file(e.g using numpy.fromfile, PIL.Image.open)
        # 2. Preprocess the data (e.g torchvision.Transform)
        # 3. Return a data pair (e.g. image and label)
        pass
    def __len__(self):
        return 0

    
# You can then use the prebuilt data loader
# custom_dataset = CustomDataset()
# train_loader = torch.utils.data.DataLoader(
#     dataset=custom_dataset,
#     batch_size=64,
#     shuffle=True
# )

### 6. Pretrained model 

In [24]:
# Download and Load ResNet-18
resnet = torchvision.models.resnet18(pretrained=True)
# 微调模型
for param in resnet.parameters():
    param.requires_grad = False
    
    
# Replace the top layer for finetuning
resnet.fc = nn.Linear(resnet.fc.in_features, 100)

# Porwad pass
images = torch.randn(64, 3, 224, 224)
outputs = resnet(images)

print(outputs.size())

C:\ProgramData\Anaconda3\lib\site-packages\torch\nn\functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  ..\c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


torch.Size([64, 100])


### 7. Save and load the model 

In [25]:
# Save and load the entire model
torch.save(resnet, 'model.ckpt')
model = torch.load('model.ckpt')

# Save and load the model parameters
torch.save(resnet.state_dict(), 'params.ckpt')
resnet.load_state_dict(torch.load('params.ckpt'))

<All keys matched successfully>